In [ ]:
#!conda install lxml 
#!conda install html5lib

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last"

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = None

In [2]:
from bs4 import BeautifulSoup as bs
import requests

In [3]:
#Get the html source code for the wikipedia page
source = requests.get('https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs').text

In [4]:
soup = bs(source, 'lxml') #decode
print(soup.prettify()) #display
#NOTE: soup.get_text() will get all the text on that page. Through some trial and error, I could isolate text indices 7274 till 29752 which were for the suburbs
#The above is a weak method because any update in the wiki page will break it down. I should ideally be fetching the suburbs in a smarter manner through tags - will improve on this

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Melbourne suburbs - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Melbourne_suburbs","wgTitle":"List of Melbourne suburbs","wgCurRevisionId":896102832,"wgRevisionId":896102832,"wgArticleId":745628,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","Use dmy dates from October 2012","Geography of Victoria (Australia)","Lists of suburbs in Australia","Suburbs of Melbourne","Melbourne-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat"

In [ ]:
#Two things being done in the next line of code
#1. Get the text -> place in df using new rows for different lines
#2. Place a '|' wherever there is a bracket so that we can use it as a delimiter without removing the bracket
wiki_text = pd.DataFrame(soup.get_text()[7274:29752].replace('(', '|(').split('\n'), columns=['Suburbs']) 


#wiki_text['Suburbs'] = wiki_text['Suburbs'].astype('str') #convert to string <- not working
wiki_text.head()
wiki_text.tail()
wiki_text.dtypes

In [ ]:
suburbs = wiki_text["Suburbs"].str.split("|", n = 1, expand = True) #split the cells using the | we placed so that Suburbs column only has suburb name
suburbs.columns = ['Suburb', 'Info'] #name the columns

suburbs['Suburb'] = suburbs['Suburb'].str.replace('[^a-zA-Z0-9]', ' ') #replace any non alpha numeric character by a space (to remove any strange characters)

suburbs.dtypes
suburbs.shape
suburbs.head(100)

In [ ]:
#If you see above, the Suburb columns have some values without a postal code. Actually these are just names are areas within the parent suburbs which is the value before it
#We need to remove those rows. We can therefore try to remove any rows where the suburb cells have no numbers i.e. no postal code.

suburbs['characs'] =suburbs.Suburb.str.replace('[^a-zA-Z0-9]','') # create a new column with just the letters and numbers in the Suburbs columns
suburbs['characs'] = suburbs['characs'].str.isalpha() #convert the column to true if it has no numbers
suburbs = suburbs[suburbs['characs']== False].drop('characs', axis = 1) #drop any rows where the characs is False i.e. it has only letters 
suburbs.head()

In [ ]:
#suburbs.describe() 
#It was seen that some Suburbs are being duplicated!! Turns out that these suburbs are shared between multiple cities. So we must remove them!
suburbs.drop_duplicates(subset='Suburb',inplace=True)
suburbs.shape

In [ ]:
#In order to ensure that the suburb column only had suburbs I decided to see the len of characters in it
suburbs['len'] = list(map(lambda x: len(x), suburbs['Suburb']))
suburbs.describe()

All the quartiles are within a tight region, but somehow the max is almost 6 times bigger than the mean! I guess there are some sentences which have survived. Let's see what they are!

In [ ]:
suburbs[suburbs['len']>25].sort_values('len',ascending=False)

Indeed, it is a sentence probably a caption to an image! This again reinstates the need to fetch data in a smarter way than just indexing the entire text of a html page! Sigh
Now, for those 44 and above, we can just remove.
For Diggers Rest, it seems like it is still a suburb but it has a string at the end. We will deal with it also.

In [ ]:
suburbs[suburbs['Suburb'].str.contains('3427')]

Well, turns out Diggers Rest is a duplicate as well!! We can just drop it as well. So keep only those rows which have character len less than 29

In [ ]:
suburbs = suburbs[suburbs['len']<29]
suburbs.shape

In [36]:
sub_list = [] #create an empty list
x=0 #create counter
for text in soup.find_all('li', class_ = None): #loop through all the <li tags without any class - these are where the suburbs are
    #print(x)
    #x +=1
    sub_list=sub_list+[text] #add suburbs to list
suburbs1 = pd.DataFrame(sub_list, columns=['Suburb']) #convert list to df
#suburbs1.head()

#suburbs1.shape

suburbs1['Suburb'] = suburbs1['Suburb'].astype(str) #change dtype to string

suburbs1[['1','2']] = suburbs1["Suburb"].str.split("<a href=", n = 1, expand = True) #split through <a href= delimiter - link after this

suburbs1[['3','4']] = suburbs1['2'].str.split("title=", n = 1, expand = True) #split across title= after which suburb name is 


suburbs1[['5','6']] = suburbs1['4'].str.split("</a> ", n = 1, expand = True) #split across <a/a tag - after which postal code is

suburbs1['7'] = suburbs1['6'].str[:4] #crop out first four characs of postal code

suburbs1.drop(['Suburb','2','4','6'], axis=1) #drop the unnecessary coloums

print(suburbs1.shape)
#suburbs1.head(5)

(1101, 8)


In [39]:
print("originally", suburbs1.shape)

suburbs1 = suburbs1.dropna(subset=['7'])
suburbs1.drop_duplicates(inplace=True, subset=['7']) #drop duplicate postal codes
print("After dropping duplicates", suburbs1.shape)

suburbs1 = suburbs1[~suburbs1['7'].str.contains("[a-zA-Z]").fillna(False)] #removing postal codes = None
print("After dropping Nones", suburbs1.shape)

non_num_index = suburbs1.index[~suburbs1['7'].str.isnumeric() ] #check for any non-numeric cell
print(non_num_index)

suburbs1.drop(index=non_num_index, inplace=True) #remove non-numeric postal code
print("After dropping non-numeric", suburbs1.shape)
suburbs1.sort_values(by=['7']).head(2)

originally (271, 8)
After dropping duplicates (271, 8)
After dropping Nones (271, 8)
Int64Index([], dtype='int64')
After dropping non-numeric (271, 8)


,Suburb,1,2,3,4,5,6,7
11,"<li><a href=""/wiki/Melbourne_City_Centre"" titl...",<li>,"""/wiki/Melbourne_City_Centre"" title=""Melbourne...","""/wiki/Melbourne_City_Centre""","""Melbourne City Centre"">Melbourne</a> 3000 (<a...","""Melbourne City Centre"">Melbourne","3000 (<a href=""/wiki/Central_business_district...",3000
7,"<li><a href=""/wiki/East_Melbourne,_Victoria"" t...",<li>,"""/wiki/East_Melbourne,_Victoria"" title=""East M...","""/wiki/East_Melbourne,_Victoria""","""East Melbourne, Victoria"">East Melbourne</a> ...","""East Melbourne, Victoria"">East Melbourne","3002\n<ul><li><a href=""/wiki/Jolimont,_Distric...",3002


Postal codes all cleaned up. Total of 271 postal codes.
Next step is to clean up the other columns

In [91]:
suburbs1['links'] = str("https://en.wikipedia.org") + suburbs1['3'].str.split("\"",expand=True)[1] #Create link using column['3']
suburbs1['Suburb'] = suburbs1['5'].str.split(",", expand= True)[0].str.replace("\"","")
suburbs1.head()

,Suburb,1,2,3,4,5,6,7,links
3,Carlton,<li>,"""/wiki/Carlton,_Victoria"" title=""Carlton, Vict...","""/wiki/Carlton,_Victoria""","""Carlton, Victoria"">Carlton</a> 3053\n<ul><li>...","""Carlton, Victoria"">Carlton",3053\n<ul><li><i>Carlton South</i></li></ul></li>,3053,"https://en.wikipedia.org/wiki/Carlton,_Victoria"
5,Carlton North,<li>,"""/wiki/Carlton_North,_Victoria"" title=""Carlton...","""/wiki/Carlton_North,_Victoria""","""Carlton North, Victoria"">Carlton North</a> 30...","""Carlton North, Victoria"">Carlton North","3054 (Shared with <a href=""#City_of_Yarra"">Cit...",3054,"https://en.wikipedia.org/wiki/Carlton_North,_V..."
6,Docklands,<li>,"""/wiki/Docklands,_Victoria"" title=""Docklands, ...","""/wiki/Docklands,_Victoria""","""Docklands, Victoria"">Docklands</a> 3008</li>","""Docklands, Victoria"">Docklands",3008</li>,3008,"https://en.wikipedia.org/wiki/Docklands,_Victoria"
7,East Melbourne,<li>,"""/wiki/East_Melbourne,_Victoria"" title=""East M...","""/wiki/East_Melbourne,_Victoria""","""East Melbourne, Victoria"">East Melbourne</a> ...","""East Melbourne, Victoria"">East Melbourne","3002\n<ul><li><a href=""/wiki/Jolimont,_Distric...",3002,"https://en.wikipedia.org/wiki/East_Melbourne,_..."
9,Flemington,<li>,"""/wiki/Flemington,_Victoria"" title=""Flemington...","""/wiki/Flemington,_Victoria""","""Flemington, Victoria"">Flemington</a> 3031 (Sh...","""Flemington, Victoria"">Flemington","3031 (Shared with <a href=""#City_of_Moonee_Val...",3031,"https://en.wikipedia.org/wiki/Flemington,_Vict..."


In [ ]:
suburbs2['7']

In [ ]:
suburbs2.reset_index(drop=True, inplace=True)

In [18]:
non_num_index = suburbs2.index[~suburbs2['7'].str.isnumeric() ] #check for any non-numeric cell
print(non_num_index)

suburbs2.drop(index=non_num_index, inplace=True) #delete that cell

Int64Index([], dtype='int64')


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
